# Dev sqlite window functions

In [ ]:
# sqlite_path = '/Users/danb/hack/birdgram/mobile/app/assets/payloads/CA100/search_recs/search_recs.sqlite3'
# sqlite_path = '/Users/danb/hack/birdgram/mobile/app/assets/payloads/CA3500/search_recs/search_recs.sqlite3'
sqlite_path = '/Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3'

from notebooks import *

db_url = f'sqlite:///{sqlite_path}'
eng = sqla.create_engine(db_url)

# For `%%sqla -Iplan`
def plan(sql):
    _plan(sql)
    return sql

def _plan(sql: str, **kwargs):
    print_sqlite_query_plan(_query(f'explain query plan {sql}', **kwargs))

# Normal python version of `%%sqla -Iplan`
def query(sql: str, **kwargs) -> DF:
    _plan(sql, **kwargs)
    return _query(sql, **kwargs)

def _query(sql: str, **kwargs) -> DF:
    return pd.read_sql(con=eng, sql=sql, **kwargs)

def print_sqlite_query_plan(df: DF) -> str:
    # Query plan structure
    root = dict(detail=None, children=[])
    by_id = {0: root}
    for row in df_rows(df):
        node = dict(detail=row.detail, children=[])
        by_id[row.id] = node
        by_id[row.parent]['children'].append(node)
    # Print (pretty)
    def f(plan):
        if not plan['detail']:
            return [f(x) for x in plan['children']]
        if not plan['children']:
            return plan['detail']
        else:
            return {plan['detail']: [f(x) for x in plan['children']]}
    print(yaml.safe_dump(default_flow_style=False, data=f(root)))

In [ ]:
%%sqla -ceng -Iplan -p3
--# TODO Example window
with
  A as (
    select *, f_preds_0 as d_pc, species as slp
    from search_recs
    where species not in ('SOSP', 'HOSP', 'HOFI', 'BEWR', 'PYNU')
  ),
  B as (
    select species, source_id, slp, d_pc, row_number() over (partition by species order by d_pc) as sp_d_pc_i
    from A
  )
select *
from B
where sp_d_pc_i <= 3
limit 20

In [ ]:
%%sqla -ceng -Iplan -p3
--# TODO (0/n): Baseline, copied from app logs
select
  S.*,
  coalesce(slp.slp, 1e38) as slp
--# Must select * from (...) else union complains about nested order by
from (select * from (
  select
    S.*,
    1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1) / S.norm_f_preds / Q.norm_f_preds as d_pc
  from search_recs S
    left join (select * from search_recs where source_id = 'xc:417354') Q on true --# Only 1 row in Q
  where true
    and S.species = 'NOWA'
    --# Empty subquery for species outside of placeFilter
    and S.quality in ('A', 'B')
    and S.source_id != 'xc:417354' --# Exclude query_rec from results
  order by
    d_pc asc
  limit 3
) union all select * from (
  select
    S.*,
    1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1) / S.norm_f_preds / Q.norm_f_preds as d_pc
  from search_recs S
    left join (select * from search_recs where source_id = 'xc:417354') Q on true --# Only 1 row in Q
  where true
    and S.species = 'WIWA'
    --# Empty subquery for species outside of placeFilter
    and S.quality in ('A', 'B')
    and S.source_id != 'xc:417354' --# Exclude query_rec from results
  order by
    d_pc asc
  limit 3
)) S
  left join (
    select column1 as species, column2 as slp from (values ('NOWA', 2.7278727231365987), ('WIWA', 2.826477946769426))
  ) slp on S.species = slp.species
order by
  slp asc,
  d_pc asc
limit 31

In [ ]:
%%sqla -ceng -p3
--# TODO (1/n)
select
  --# S.*,
  S.species,
  S.source_id,
  coalesce(slp.slp, 1e38) as slp,
  S.d_pc,
  --# row_number() over (partition by species order by d_pc) as sp_d_pc_i,
  null
from (
  select * from (
    select
      S.*,
      1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1) / S.norm_f_preds / Q.norm_f_preds as d_pc
    from search_recs S
      left join (select * from search_recs where source_id = 'xc:417354') Q on true --# Only 1 row in Q
    where true
      and S.species = 'NOWA'
      --# Empty subquery for species outside of placeFilter
      and S.quality in ('A', 'B')
      and S.source_id != 'xc:417354' --# Exclude query_rec from results
    order by
      d_pc asc
    limit 3
  ) union all select * from (
    select
      S.*,
      1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1) / S.norm_f_preds / Q.norm_f_preds as d_pc
    from search_recs S
      left join (select * from search_recs where source_id = 'xc:417354') Q on true --# Only 1 row in Q
    where true
      and S.species = 'WIWA'
      --# Empty subquery for species outside of placeFilter
      and S.quality in ('A', 'B')
      and S.source_id != 'xc:417354' --# Exclude query_rec from results
    order by
      d_pc asc
    limit 3
  )
) S left join (
  select column1 as species, column2 as slp
  from (values ('NOWA', 2.7278727231365987), ('WIWA', 2.826477946769426))
) slp on S.species = slp.species
order by slp asc, d_pc asc
limit 31

In [ ]:
%%sqla -ceng -q
--# TODO (2/n)
select
  S.species,
  S.source_id,
  coalesce(slp.slp, 1e38) as slp,
  S.d_pc,
  --# row_number() over (partition by species order by d_pc) as sp_d_pc_i,
  null
from (
  select * from (
    select
      S.*,
      1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1) / S.norm_f_preds / Q.norm_f_preds as d_pc
    from search_recs S
      left join (select * from search_recs where source_id = 'xc:417354') Q on true
    where true
      and S.species = 'NOWA'
      and S.quality in ('A', 'B')
      and S.source_id != 'xc:417354' --# Exclude query_rec
    order by d_pc asc
    limit 3
  ) union all select * from (
    select
      S.*,
      1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1) / S.norm_f_preds / Q.norm_f_preds as d_pc
    from search_recs S
      left join (select * from search_recs where source_id = 'xc:417354') Q on true
    where true
      and S.species = 'WIWA'
      and S.quality in ('A', 'B')
      and S.source_id != 'xc:417354' --# Exclude query_rec
    order by d_pc asc
    limit 3
  )
) S
  left join (select column1 as species, column2 as slp from (values
    ('NOWA', 2.7278727231365987),
    ('WIWA', 2.826477946769426)
  )) slp on S.species = slp.species
order by slp asc, d_pc asc
limit 31

In [ ]:
%%sqla -ceng -q
--# TODO (3/n)
select
  S.species,
  S.source_id,
  coalesce(slp.slp, 1e38) as slp,
  S.d_pc,
  sp_d_pc_i
from (
  select
    S.*,
    row_number() over (partition by S.species order by d_pc) as sp_d_pc_i
  from (
    select
      S.*,
      1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1) / S.norm_f_preds / Q.norm_f_preds as d_pc
    from search_recs S
      left join (select * from search_recs where source_id = 'xc:417354') Q on true --# For dot(S.f_preds, Q.f_preds)
    where true
      and S.species in ('NOWA', 'WIWA') --# [topSlps.map(x => x.species)]
      and S.quality in ('A', 'B')
      and S.source_id != 'xc:417354' --# Exclude query_rec
    order by d_pc asc
  ) S
) S
  --# TODO Perf: try doing both joins at once, above
  left join (select column1 as species, column2 as slp from (values
    ('NOWA', 2.7278727231365987), --# [topSlps (-> tableSlp)]
    ('WIWA', 2.826477946769426)
  )) slp on S.species = slp.species
where sp_d_pc_i <= 3 --# [n_per_sp]
order by slp asc, d_pc asc
limit 31 --# [n_recs]

In [ ]:
%%sqla -ceng -p3 -Iplan
--# TODO (4/n)
select
  species,
  source_id,
  coalesce(slp, 1e38) as slp,
  d_pc,
  sp_d_pc_i
from (
  select
    *,
    row_number() over (partition by species order by d_pc) as sp_d_pc_i
  from (
    select
      S.*,
      slp.slp,
      1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1) / S.norm_f_preds / Q.norm_f_preds as d_pc
    from search_recs S
      join (select * from search_recs where source_id = 'xc:417354') Q on true --# For dot(S.f_preds, Q.f_preds)
      join (select column1 as species, column2 as slp from (values
        ('NOWA', 2.7278727231365987), --# [topSlps (-> tableSlp)]
        ('WIWA', 2.826477946769426)
      )) slp on S.species = slp.species
    where true
      and S.quality in ('A', 'B')
      and S.source_id != 'xc:417354' --# Exclude query_rec
  )
)
where sp_d_pc_i <= 3 --# [n_per_sp]
order by slp asc, d_pc asc
limit 31 --# [n_recs]

In [ ]:
%%sqla -ceng -p3 -Iplan
--# TODO (5/n): New baseline (copied from app logs)
select
  S.species, S.source_id,
  sp_d_pc_i,
  coalesce(slp.slp, 1e38) as slp,
  S.d_pc
  --# S.variable, S.bird_seen, S.speed, S.remarks, S.hour, S.species_com_name, S.taxon_order, S.time, S.species_species_group, S.quality, S.id, S.license, S.elevation, S.lng, S.species, S.background_species, S.background, S.license_detail, S.duration_s, S.samples_n, S.place, S.n_background_species, S.date, S.species_taxon_order, S.year, S.xc_channels, S.downloaded, S.samples_mb, S.volume, S.number_of_notes, S.license_type, S.xc_bitrate_of_mp3, S.species_longhand, S.species_sci_name, S.pitch, S.month_day, S.path, S.url, S.norm_f_preds, S.species_query, S.state, S.channels, S.basename, S.species_family, S.species_taxon_id, S.type, S.recs_for_sp, S.country, S.lat, S.species_order, S.sample_width_bit, S.subspecies, S.download, S.source_id, S.taxon_id, S.recordist, S.place_only, S.shorthand, S.length, S.xc_sampling_rate, S.month, S.xc_id, S.state_only, S.dataset, S.playback_used, S.sample_rate, S.locality
from (
  select
    S.*,
    row_number() over (partition by S.species order by S.d_pc) as sp_d_pc_i
  from (
    select
      S.*,
      1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1 + S.f_preds_2*Q.f_preds_2 + S.f_preds_3*Q.f_preds_3 + S.f_preds_4*Q.f_preds_4 + S.f_preds_5*Q.f_preds_5 + S.f_preds_6*Q.f_preds_6 + S.f_preds_7*Q.f_preds_7 + S.f_preds_8*Q.f_preds_8 + S.f_preds_9*Q.f_preds_9 + S.f_preds_10*Q.f_preds_10 + S.f_preds_11*Q.f_preds_11 + S.f_preds_12*Q.f_preds_12 + S.f_preds_13*Q.f_preds_13 + S.f_preds_14*Q.f_preds_14 + S.f_preds_15*Q.f_preds_15 + S.f_preds_16*Q.f_preds_16 + S.f_preds_17*Q.f_preds_17 + S.f_preds_18*Q.f_preds_18 + S.f_preds_19*Q.f_preds_19 + S.f_preds_20*Q.f_preds_20 + S.f_preds_21*Q.f_preds_21 + S.f_preds_22*Q.f_preds_22 + S.f_preds_23*Q.f_preds_23 + S.f_preds_24*Q.f_preds_24 + S.f_preds_25*Q.f_preds_25 + S.f_preds_26*Q.f_preds_26 + S.f_preds_27*Q.f_preds_27 + S.f_preds_28*Q.f_preds_28 + S.f_preds_29*Q.f_preds_29 + S.f_preds_30*Q.f_preds_30 + S.f_preds_31*Q.f_preds_31 + S.f_preds_32*Q.f_preds_32 + S.f_preds_33*Q.f_preds_33 + S.f_preds_34*Q.f_preds_34 + S.f_preds_35*Q.f_preds_35 + S.f_preds_36*Q.f_preds_36 + S.f_preds_37*Q.f_preds_37 + S.f_preds_38*Q.f_preds_38 + S.f_preds_39*Q.f_preds_39 + S.f_preds_40*Q.f_preds_40 + S.f_preds_41*Q.f_preds_41 + S.f_preds_42*Q.f_preds_42 + S.f_preds_43*Q.f_preds_43 + S.f_preds_44*Q.f_preds_44 + S.f_preds_45*Q.f_preds_45 + S.f_preds_46*Q.f_preds_46 + S.f_preds_47*Q.f_preds_47 + S.f_preds_48*Q.f_preds_48 + S.f_preds_49*Q.f_preds_49 + S.f_preds_50*Q.f_preds_50 + S.f_preds_51*Q.f_preds_51 + S.f_preds_52*Q.f_preds_52 + S.f_preds_53*Q.f_preds_53 + S.f_preds_54*Q.f_preds_54 + S.f_preds_55*Q.f_preds_55 + S.f_preds_56*Q.f_preds_56 + S.f_preds_57*Q.f_preds_57 + S.f_preds_58*Q.f_preds_58 + S.f_preds_59*Q.f_preds_59 + S.f_preds_60*Q.f_preds_60 + S.f_preds_61*Q.f_preds_61 + S.f_preds_62*Q.f_preds_62 + S.f_preds_63*Q.f_preds_63 + S.f_preds_64*Q.f_preds_64 + S.f_preds_65*Q.f_preds_65 + S.f_preds_66*Q.f_preds_66 + S.f_preds_67*Q.f_preds_67 + S.f_preds_68*Q.f_preds_68 + S.f_preds_69*Q.f_preds_69 + S.f_preds_70*Q.f_preds_70 + S.f_preds_71*Q.f_preds_71 + S.f_preds_72*Q.f_preds_72 + S.f_preds_73*Q.f_preds_73 + S.f_preds_74*Q.f_preds_74 + S.f_preds_75*Q.f_preds_75 + S.f_preds_76*Q.f_preds_76 + S.f_preds_77*Q.f_preds_77 + S.f_preds_78*Q.f_preds_78 + S.f_preds_79*Q.f_preds_79 + S.f_preds_80*Q.f_preds_80 + S.f_preds_81*Q.f_preds_81 + S.f_preds_82*Q.f_preds_82 + S.f_preds_83*Q.f_preds_83 + S.f_preds_84*Q.f_preds_84 + S.f_preds_85*Q.f_preds_85 + S.f_preds_86*Q.f_preds_86 + S.f_preds_87*Q.f_preds_87 + S.f_preds_88*Q.f_preds_88 + S.f_preds_89*Q.f_preds_89 + S.f_preds_90*Q.f_preds_90 + S.f_preds_91*Q.f_preds_91 + S.f_preds_92*Q.f_preds_92 + S.f_preds_93*Q.f_preds_93 + S.f_preds_94*Q.f_preds_94 + S.f_preds_95*Q.f_preds_95 + S.f_preds_96*Q.f_preds_96 + S.f_preds_97*Q.f_preds_97 + S.f_preds_98*Q.f_preds_98 + S.f_preds_99*Q.f_preds_99 + S.f_preds_100*Q.f_preds_100 + S.f_preds_101*Q.f_preds_101 + S.f_preds_102*Q.f_preds_102 + S.f_preds_103*Q.f_preds_103 + S.f_preds_104*Q.f_preds_104 + S.f_preds_105*Q.f_preds_105 + S.f_preds_106*Q.f_preds_106 + S.f_preds_107*Q.f_preds_107 + S.f_preds_108*Q.f_preds_108 + S.f_preds_109*Q.f_preds_109 + S.f_preds_110*Q.f_preds_110 + S.f_preds_111*Q.f_preds_111 + S.f_preds_112*Q.f_preds_112 + S.f_preds_113*Q.f_preds_113 + S.f_preds_114*Q.f_preds_114 + S.f_preds_115*Q.f_preds_115 + S.f_preds_116*Q.f_preds_116 + S.f_preds_117*Q.f_preds_117 + S.f_preds_118*Q.f_preds_118 + S.f_preds_119*Q.f_preds_119 + S.f_preds_120*Q.f_preds_120 + S.f_preds_121*Q.f_preds_121 + S.f_preds_122*Q.f_preds_122 + S.f_preds_123*Q.f_preds_123 + S.f_preds_124*Q.f_preds_124 + S.f_preds_125*Q.f_preds_125 + S.f_preds_126*Q.f_preds_126 + S.f_preds_127*Q.f_preds_127 + S.f_preds_128*Q.f_preds_128 + S.f_preds_129*Q.f_preds_129 + S.f_preds_130*Q.f_preds_130 + S.f_preds_131*Q.f_preds_131 + S.f_preds_132*Q.f_preds_132 + S.f_preds_133*Q.f_preds_133 + S.f_preds_134*Q.f_preds_134 + S.f_preds_135*Q.f_preds_135 + S.f_preds_136*Q.f_preds_136 + S.f_preds_137*Q.f_preds_137 + S.f_preds_138*Q.f_preds_138 + S.f_preds_139*Q.f_preds_139 + S.f_preds_140*Q.f_preds_140 + S.f_preds_141*Q.f_preds_141 + S.f_preds_142*Q.f_preds_142 + S.f_preds_143*Q.f_preds_143 + S.f_preds_144*Q.f_preds_144 + S.f_preds_145*Q.f_preds_145 + S.f_preds_146*Q.f_preds_146 + S.f_preds_147*Q.f_preds_147 + S.f_preds_148*Q.f_preds_148 + S.f_preds_149*Q.f_preds_149 + S.f_preds_150*Q.f_preds_150 + S.f_preds_151*Q.f_preds_151 + S.f_preds_152*Q.f_preds_152 + S.f_preds_153*Q.f_preds_153 + S.f_preds_154*Q.f_preds_154 + S.f_preds_155*Q.f_preds_155 + S.f_preds_156*Q.f_preds_156 + S.f_preds_157*Q.f_preds_157 + S.f_preds_158*Q.f_preds_158 + S.f_preds_159*Q.f_preds_159 + S.f_preds_160*Q.f_preds_160 + S.f_preds_161*Q.f_preds_161 + S.f_preds_162*Q.f_preds_162 + S.f_preds_163*Q.f_preds_163 + S.f_preds_164*Q.f_preds_164 + S.f_preds_165*Q.f_preds_165 + S.f_preds_166*Q.f_preds_166 + S.f_preds_167*Q.f_preds_167 + S.f_preds_168*Q.f_preds_168 + S.f_preds_169*Q.f_preds_169 + S.f_preds_170*Q.f_preds_170 + S.f_preds_171*Q.f_preds_171 + S.f_preds_172*Q.f_preds_172 + S.f_preds_173*Q.f_preds_173 + S.f_preds_174*Q.f_preds_174 + S.f_preds_175*Q.f_preds_175 + S.f_preds_176*Q.f_preds_176 + S.f_preds_177*Q.f_preds_177 + S.f_preds_178*Q.f_preds_178 + S.f_preds_179*Q.f_preds_179 + S.f_preds_180*Q.f_preds_180 + S.f_preds_181*Q.f_preds_181 + S.f_preds_182*Q.f_preds_182 + S.f_preds_183*Q.f_preds_183 + S.f_preds_184*Q.f_preds_184 + S.f_preds_185*Q.f_preds_185 + S.f_preds_186*Q.f_preds_186 + S.f_preds_187*Q.f_preds_187 + S.f_preds_188*Q.f_preds_188 + S.f_preds_189*Q.f_preds_189 + S.f_preds_190*Q.f_preds_190 + S.f_preds_191*Q.f_preds_191 + S.f_preds_192*Q.f_preds_192 + S.f_preds_193*Q.f_preds_193 + S.f_preds_194*Q.f_preds_194 + S.f_preds_195*Q.f_preds_195 + S.f_preds_196*Q.f_preds_196 + S.f_preds_197*Q.f_preds_197 + S.f_preds_198*Q.f_preds_198 + S.f_preds_199*Q.f_preds_199 + S.f_preds_200*Q.f_preds_200 + S.f_preds_201*Q.f_preds_201 + S.f_preds_202*Q.f_preds_202 + S.f_preds_203*Q.f_preds_203 + S.f_preds_204*Q.f_preds_204 + S.f_preds_205*Q.f_preds_205 + S.f_preds_206*Q.f_preds_206 + S.f_preds_207*Q.f_preds_207 + S.f_preds_208*Q.f_preds_208 + S.f_preds_209*Q.f_preds_209 + S.f_preds_210*Q.f_preds_210 + S.f_preds_211*Q.f_preds_211 + S.f_preds_212*Q.f_preds_212 + S.f_preds_213*Q.f_preds_213 + S.f_preds_214*Q.f_preds_214 + S.f_preds_215*Q.f_preds_215 + S.f_preds_216*Q.f_preds_216 + S.f_preds_217*Q.f_preds_217 + S.f_preds_218*Q.f_preds_218 + S.f_preds_219*Q.f_preds_219 + S.f_preds_220*Q.f_preds_220 + S.f_preds_221*Q.f_preds_221 + S.f_preds_222*Q.f_preds_222 + S.f_preds_223*Q.f_preds_223 + S.f_preds_224*Q.f_preds_224 + S.f_preds_225*Q.f_preds_225 + S.f_preds_226*Q.f_preds_226 + S.f_preds_227*Q.f_preds_227 + S.f_preds_228*Q.f_preds_228 + S.f_preds_229*Q.f_preds_229 + S.f_preds_230*Q.f_preds_230 + S.f_preds_231*Q.f_preds_231 + S.f_preds_232*Q.f_preds_232 + S.f_preds_233*Q.f_preds_233 + S.f_preds_234*Q.f_preds_234 + S.f_preds_235*Q.f_preds_235 + S.f_preds_236*Q.f_preds_236 + S.f_preds_237*Q.f_preds_237 + S.f_preds_238*Q.f_preds_238 + S.f_preds_239*Q.f_preds_239 + S.f_preds_240*Q.f_preds_240 + S.f_preds_241*Q.f_preds_241 + S.f_preds_242*Q.f_preds_242 + S.f_preds_243*Q.f_preds_243 + S.f_preds_244*Q.f_preds_244 + S.f_preds_245*Q.f_preds_245 + S.f_preds_246*Q.f_preds_246 + S.f_preds_247*Q.f_preds_247 + S.f_preds_248*Q.f_preds_248 + S.f_preds_249*Q.f_preds_249 + S.f_preds_250*Q.f_preds_250 + S.f_preds_251*Q.f_preds_251 + S.f_preds_252*Q.f_preds_252 + S.f_preds_253*Q.f_preds_253 + S.f_preds_254*Q.f_preds_254 + S.f_preds_255*Q.f_preds_255 + S.f_preds_256*Q.f_preds_256 + S.f_preds_257*Q.f_preds_257 + S.f_preds_258*Q.f_preds_258 + S.f_preds_259*Q.f_preds_259 + S.f_preds_260*Q.f_preds_260 + S.f_preds_261*Q.f_preds_261 + S.f_preds_262*Q.f_preds_262 + S.f_preds_263*Q.f_preds_263 + S.f_preds_264*Q.f_preds_264 + S.f_preds_265*Q.f_preds_265 + S.f_preds_266*Q.f_preds_266 + S.f_preds_267*Q.f_preds_267 + S.f_preds_268*Q.f_preds_268 + S.f_preds_269*Q.f_preds_269 + S.f_preds_270*Q.f_preds_270 + S.f_preds_271*Q.f_preds_271 + S.f_preds_272*Q.f_preds_272 + S.f_preds_273*Q.f_preds_273 + S.f_preds_274*Q.f_preds_274 + S.f_preds_275*Q.f_preds_275 + S.f_preds_276*Q.f_preds_276 + S.f_preds_277*Q.f_preds_277 + S.f_preds_278*Q.f_preds_278 + S.f_preds_279*Q.f_preds_279 + S.f_preds_280*Q.f_preds_280 + S.f_preds_281*Q.f_preds_281 + S.f_preds_282*Q.f_preds_282 + S.f_preds_283*Q.f_preds_283 + S.f_preds_284*Q.f_preds_284 + S.f_preds_285*Q.f_preds_285 + S.f_preds_286*Q.f_preds_286 + S.f_preds_287*Q.f_preds_287 + S.f_preds_288*Q.f_preds_288 + S.f_preds_289*Q.f_preds_289 + S.f_preds_290*Q.f_preds_290 + S.f_preds_291*Q.f_preds_291 + S.f_preds_292*Q.f_preds_292 + S.f_preds_293*Q.f_preds_293 + S.f_preds_294*Q.f_preds_294 + S.f_preds_295*Q.f_preds_295 + S.f_preds_296*Q.f_preds_296 + S.f_preds_297*Q.f_preds_297 + S.f_preds_298*Q.f_preds_298 + S.f_preds_299*Q.f_preds_299 + S.f_preds_300*Q.f_preds_300 + S.f_preds_301*Q.f_preds_301 + S.f_preds_302*Q.f_preds_302 + S.f_preds_303*Q.f_preds_303 + S.f_preds_304*Q.f_preds_304 + S.f_preds_305*Q.f_preds_305 + S.f_preds_306*Q.f_preds_306 + S.f_preds_307*Q.f_preds_307 + S.f_preds_308*Q.f_preds_308 + S.f_preds_309*Q.f_preds_309 + S.f_preds_310*Q.f_preds_310 + S.f_preds_311*Q.f_preds_311 + S.f_preds_312*Q.f_preds_312 + S.f_preds_313*Q.f_preds_313 + S.f_preds_314*Q.f_preds_314 + S.f_preds_315*Q.f_preds_315 + S.f_preds_316*Q.f_preds_316 + S.f_preds_317*Q.f_preds_317 + S.f_preds_318*Q.f_preds_318 + S.f_preds_319*Q.f_preds_319 + S.f_preds_320*Q.f_preds_320 + S.f_preds_321*Q.f_preds_321 + S.f_preds_322*Q.f_preds_322 + S.f_preds_323*Q.f_preds_323 + S.f_preds_324*Q.f_preds_324 + S.f_preds_325*Q.f_preds_325 + S.f_preds_326*Q.f_preds_326 + S.f_preds_327*Q.f_preds_327 + S.f_preds_328*Q.f_preds_328 + S.f_preds_329*Q.f_preds_329 + S.f_preds_330*Q.f_preds_330) / S.norm_f_preds / Q.norm_f_preds as d_pc
    from search_recs S
      left join (select * from search_recs where source_id = 'xc:416410') Q on true --# 1 row, for dot(S,Q)
    where true
      --# Empty subquery for species outside of placeFilter
      and S.quality in ('A', 'B')
      and S.source_id != 'xc:416410' --# Exclude query_rec from results
  ) S
) S
  left join (
    select column1 as species, column2 as slp from (values ('BRCR', 2.531013988420692), ('BEWR', 2.532770076870172), ('LISP', 2.612302265470383), ('OATI', 2.6209269915633775), ('YBCH', 2.693449148368873), ('RWBL', 2.85915840716264), ('YHBL', 2.9407066815495098), ('JUTI', 3.1964930770861226), ('SOSP', 3.3536266585633028), ('EUST', 3.4345511861796005), ('WIFL', 3.4793422717445033), ('ABTO', 3.496172099469417), ('GRVI', 3.5193633517510596), ('RCKI', 3.6404688761047668), ('LASP', 3.6512290161829797), ('GTGR', 3.9715303202185503), ('WEVI', 4.211134790180211), ('LOSH', 4.242274068412453), ('MAWA', 4.350559483605008), ('WIWR', 4.367698139109152), ('MOWA', 4.452379012375921), ('SWTH', 4.460531357394325), ('HUVI', 4.645053683966041), ('CBTH', 4.686734006282624), ('GCSP', 4.706664237639098), ('COYE', 4.738316347112597), ('SUTA', 4.768478476743198), ('CACW', 4.897283943714701), ('BTGN', 5.055958148609322), ('CITE', 5.13783401208553), ('PARE', 5.1870673093210415), ('SPTO', 5.289424823342004), ('HOWR', 5.313536619683756), ('VIRA', 5.359643554296269), ('NOMO', 5.435658780172663), ('STJA', 5.443618350266081), ('NOWA', 5.571608467440403), ('TOWA', 5.624281844181796), ('GTTO', 5.672155589635655), ('HOOR', 5.754228876946757), ('BRBL', 5.803098377199502), ('BUOR', 5.860230662760299), ('WETA', 5.870270129701547), ('SAVS', 5.9350801202552175), ('GRRO', 6.088533962280162), ('CASJ', 6.16486681026883), ('EUCD', 6.263018618968476), ('BHCO', 6.2676917297004735), ('GAQU', 6.277149956237753), ('PRAW', 6.322347171125688), ('MAWR', 6.394385748801781), ('BLTE', 6.495218547138002), ('LEBI', 6.50714035895642), ('HOGR', 6.519299953142001), ('GRHE', 6.594901668884092), ('RIRA', 6.778172676625919), ('CEDW', 6.792401744450868), ('BEVI', 6.8168670075358735), ('AMRE', 6.8434601571283), ('PUFI', 6.8800714610394955), ('TRES', 6.932283234725144), ('CAKI', 6.98838225278954), ('LBWO', 7.0568784274031495), ('MALL', 7.058447717494623), ('ISSJ', 7.094417567384517), ('GRCA', 7.104691391888663), ('DOWO', 7.233165856131655), ('RTHA', 7.320819893142206), ('REVI', 7.323189558427389), ('CANW', 7.340249438571233), ('FOSP', 7.341033665941331), ('BTSP', 7.379653544270154), ('CAGU', 7.4009806956460285), ('BCSP', 7.450540318791626), ('CLGR', 7.529309001424839), ('CALT', 7.665288650261056), ('AMBI', 7.670159206932208), ('WCSP', 7.677722963732932), ('CONI', 7.7165008502646755), ('WISA', 7.737650989254115), ('SCOR', 7.7756358010811), ('ROWR', 7.800027821612021), ('VEFL', 7.836716430514252), ('LUWA', 7.861275711075517), ('YGVI', 8.032603594833692), ('EAPH', 8.045889673331839), ('RECR', 8.054306272386695), ('PLVI', 8.0573258089993), ('VGSW', 8.150851135605016), ('DEJU', 8.177422672877068), ('WEKI', 8.192093961397726), ('AMRO', 8.207358641896416), ('COKI', 8.22691108048428), ('WOSJ', 8.23519637345769), ('TBKI', 8.248404895133326), ('BRSP', 8.260137874042956), ('GRAJ', 8.319454820372373), ('CLNU', 8.41210360706917), ('HEWA', 8.443646487477599), ('YBCU', 8.455966185801852), ('GBHE', 8.470547027359219), ('COBH', 8.473457680601122), ('WOTH', 8.47425251776894), ('CORA', 8.504145101647248), ('WFIB', 8.558195246726559), ('GCKI', 8.675845206307086), ('SNEG', 8.71489775217107), ('CAVI', 8.824804994737796), ('BCFL', 8.865598141201284), ('GADW', 8.881805163459138), ('PBGR', 8.883883139405508), ('OCWA', 8.91571997772636), ('BGGN', 8.979557632540418), ('PISI', 9.013475155286066), ('BHGR', 9.048014026311973), ('RNPH', 9.106074085564245), ('CAGN', 9.194456615444194), ('WESA', 9.253091637619606), ('HOSP', 9.329810940921154), ('YBMA', 9.331748009232133), ('HOFI', 9.352768888613367), ('WEGR', 9.36361691338387), ('ATFL', 9.395993001431028), ('OSFL', 9.43129582619472), ('BANO', 9.440415528979932), ('BARS', 9.468711682387298), ('NRWS', 9.483742792319145), ('DUFL', 9.494884209667863), ('BLGR', 9.506186466177965), ('WREN', 9.588832680404476), ('WHIM', 9.59023830590102), ('RCSP', 9.612597665765831), ('BCHU', 9.636988913398117), ('RNGR', 9.649324364699009), ('NAWA', 9.65689968046352), ('WEWP', 9.677391716081651), ('GIWO', 9.683855960858526), ('CAQU', 9.68931968940774), ('TRBL', 9.69457759244987), ('AMPI', 9.705954469663716), ('CATH', 9.756499339000824), ('FOTE', 9.786710739159583), ('INDO', 9.798281775522625), ('MGWA', 9.850680760487974), ('WAVI', 9.87826785453523), ('INBU', 9.973553500978726), ('BNST', 10.003575107070436), ('GWTE', 10.004672621664334), ('CATE', 10.024939059789673), ('COCK', 10.058411906242194), ('BRAN', 10.070341811872169), ('LBCU', 10.07268529924788), ('BEKI', 10.08892382643895), ('GREG', 10.12263126882629), ('GRFL', 10.225121670747903), ('MOCH', 10.288602435960087), ('AMAV', 10.289414125141299), ('NOHA', 10.296462703885892), ('VERD', 10.359538873224341), ('RWBU', 10.38093207690275), ('BTAH', 10.444779583769158), ('BOGU', 10.459158320788868), ('BTPI', 10.46969181032973), ('SATH', 10.494303412559143), ('EWPW', 10.499739379193759), ('BBWA', 10.528161986296945), ('AMCR', 10.534259580501928), ('ACWO', 10.556080342422899), ('LETE', 10.676353186733806), ('HETH', 10.703737887764852), ('YRWA', 10.740663773204677), ('BETH', 10.75306310082828), ('BESP', 10.775242600307786), ('JAWE', 10.79885071045656), ('LBDO', 10.881159712750236), ('CASP', 10.881791795833225), ('PIWA', 10.917633353380774), ('LENI', 11.001262318008), ('WIWA', 11.076173104920157), ('ELTE', 11.134156481436278), ('BTYW', 11.159490114984676), ('MEGU', 11.19561006888942), ('COGA', 11.234191400659466), ('SBMU', 11.272577125505444), ('GWFG', 11.283658145820533), ('HOLA', 11.289166873118502), ('YHPA', 11.30167898908479), ('BLPH', 11.328764066954324), ('COHA', 11.399349884857482), ('GRSP', 11.426447575635958), ('ROSA', 11.514579864617541), ('WHWO', 11.554579673748899), ('BCNH', 11.616288770600107), ('WEWA', 11.63805110813809), ('COGD', 11.660989309708581), ('WTSP', 11.705153274178675), ('YFGU', 11.710106064272063), ('RMPA', 11.745186647054975), ('WTKI', 11.745444015371856), ('DCFL', 11.775270191659672), ('SPSA', 11.8079276487243), ('WILL', 11.820715335632878), ('ANHU', 11.874078168204854), ('RBNU', 11.951802966228469), ('HAWO', 12.002295226387616), ('LEFL', 12.040031240389034), ('LAZB', 12.07708553255923), ('NAPA', 12.081497967472878), ('CSWA', 12.145381176599757), ('SOGR', 12.202756337377206), ('LAGO', 12.2319360538941), ('HAFL', 12.253908421142965), ('SOSA', 12.280515337039642), ('HOWA', 12.309078288331046), ('CAFI', 12.342163223335955), ('REDH', 12.342852696646961), ('CAHU', 12.34437175640791), ('PIGR', 12.370777588356646), ('AMCO', 12.371449846832995), ('CRTH', 12.40621727101908), ('CCSP', 12.418824267906743), ('WEME', 12.461447585447527), ('NUWO', 12.494546256185368), ('WBNU', 12.550259198156121), ('AMKE', 12.5879705692463), ('BFAL', 12.599010072947978), ('SUSC', 12.607163481842957), ('LEGO', 12.610194266715151), ('EGGO', 12.717279991381758), ('BTMJ', 12.734900484409916), ('LCPA', 12.785958480665037), ('SAPH', 12.85096814906219), ('BBWO', 12.853764320741432), ('TOSO', 12.887230564749972), ('PSFL', 12.887776107638214), ('BUSH', 12.89277208362413), ('WITU', 12.947646911113521), ('SNPL', 12.950877795143759), ('PAWR', 13.015895644703479), ('EVGR', 13.030762053934088), ('AMWI', 13.092302802697999), ('RBSA', 13.092911293379633), ('PTWH', 13.151428283688343), ('RBGR', 13.23211415043763), ('TRKI', 13.35721436984883), ('SEPL', 13.39987526822362), ('RCPA', 13.534886516815417), ('NOCA', 13.557554884405473), ('BLTU', 13.641568590587058), ('MAGO', 13.663751672726363), ('PHAI', 13.679443332396383), ('CANG', 13.712275984543163), ('PIJA', 13.779124742830192), ('WWDO', 13.93986752648307), ('NOFL', 13.949757055334633), ('NOPA', 13.967616353635064), ('FLOW', 14.046397653533175), ('AMGO', 14.09764953376981), ('MOUQ', 14.180958040833765), ('GRWA', 14.244986224414498), ('RTLO', 14.427145225017854), ('SACR', 14.49586095683052), ('LCTH', 14.535235838532195), ('RTPI', 14.566881452855002), ('COHU', 14.658086425044775), ('WTSW', 14.69155469601567), ('VATH', 14.760312386336937), ('CACG', 14.8905103260917), ('WEBL', 14.919877548339713), ('RRPA', 14.92159708600756), ('RSHA', 15.02440736450781), ('ROYT', 15.034523996218242), ('MWPW', 15.101850408480322), ('DUNL', 15.112934844811912), ('CLSW', 15.228457857956569), ('CHSP', 15.325009416549817), ('MERL', 15.36083794907528), ('SWHA', 15.483503064294434), ('SORA', 15.504577349038902), ('AFCD', 15.557755066107376), ('COLO', 15.571589072540275), ('PUMA', 15.580846337217782), ('GHOW', 15.592303703808632), ('LALO', 15.606447644743367), ('RBGU', 15.626537739256971), ('VASW', 15.65960731659203), ('DCCO', 15.705802157823962), ('CBCH', 15.724589490413988), ('PYNU', 15.742447906804415), ('GWGU', 16.06143149450292), ('PAGP', 16.112702834354685), ('SURF', 16.159712354031832), ('MOBL', 16.38585694486965), ('MODO', 16.57150675239372), ('PEFA', 16.61430653763232), ('SABS', 16.619973334139644), ('yecpar', 16.800916569278108), ('ALHU', 16.815754768941343), ('LEOW', 16.848703315122915), ('VESP', 16.867215051072662), ('PECO', 16.904050667056193), ('BUOW', 16.982252032483235), ('WEGU', 17.306432365345316), ('SBDO', 17.61577088023981), ('BLRA', 17.704378641694433), ('SNGO', 17.896372239524084), ('OSPR', 17.91787001874239), ('COPO', 18.000715626979783), ('DUWA', 18.28735100533962), ('PIWO', 18.399724416044027), ('GBTE', 18.476700211882985), ('BUFF', 18.570837432539953), ('KILL', 18.62114372213435), ('HEEG', 18.641071653818972), ('BLSK', 18.673321079568804), ('RODO', 18.96827923990534), ('NSWO', 19.09820594164006), ('WATA', 19.209856327280047), ('BLOY', 19.4653992890448), ('GRYE', 19.48918754998192), ('LESA', 19.491976161971518), ('LAGU', 19.787924146071667), ('AMDI', 19.90295247329546), ('MAMU', 20.236128093649693), ('WESO', 20.6932070454643), ('COMU', 21.32292012451962), ('SPOW', 22.169901163965974), ('ELOW', 22.460205414042218), ('TUSW', 22.492817124183066), ('BBPL', 22.779129209772936), ('BRAC', 23.61234120289399))
  ) slp on S.species = slp.species
where
  sp_d_pc_i <= 3
order by
  slp asc, d_pc asc
limit 31

In [ ]:
%%sqla -ceng -p3 -Iplan
--# TODO (6/n): Speed up slow perf (~20s US, ~1s CA3500)
select
  S.species, S.source_id,
  sp_d_pc_i,
  coalesce(S.slp, 1e38) as slp,
  S.d_pc
  --# S.variable, S.bird_seen, S.speed, S.remarks, S.hour, S.species_com_name, S.taxon_order, S.time, S.species_species_group, S.quality, S.id, S.license, S.elevation, S.lng, S.species, S.background_species, S.background, S.license_detail, S.duration_s, S.samples_n, S.place, S.n_background_species, S.date, S.species_taxon_order, S.year, S.xc_channels, S.downloaded, S.samples_mb, S.volume, S.number_of_notes, S.license_type, S.xc_bitrate_of_mp3, S.species_longhand, S.species_sci_name, S.pitch, S.month_day, S.path, S.url, S.norm_f_preds, S.species_query, S.state, S.channels, S.basename, S.species_family, S.species_taxon_id, S.type, S.recs_for_sp, S.country, S.lat, S.species_order, S.sample_width_bit, S.subspecies, S.download, S.source_id, S.taxon_id, S.recordist, S.place_only, S.shorthand, S.length, S.xc_sampling_rate, S.month, S.xc_id, S.state_only, S.dataset, S.playback_used, S.sample_rate, S.locality
from (
  select
    S.*,
    row_number() over (partition by S.species order by S.d_pc) as sp_d_pc_i
  from (
    select
      S.*,
      slp.slp,
      1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1 + S.f_preds_2*Q.f_preds_2 + S.f_preds_3*Q.f_preds_3 + S.f_preds_4*Q.f_preds_4 + S.f_preds_5*Q.f_preds_5 + S.f_preds_6*Q.f_preds_6 + S.f_preds_7*Q.f_preds_7 + S.f_preds_8*Q.f_preds_8 + S.f_preds_9*Q.f_preds_9 + S.f_preds_10*Q.f_preds_10 + S.f_preds_11*Q.f_preds_11 + S.f_preds_12*Q.f_preds_12 + S.f_preds_13*Q.f_preds_13 + S.f_preds_14*Q.f_preds_14 + S.f_preds_15*Q.f_preds_15 + S.f_preds_16*Q.f_preds_16 + S.f_preds_17*Q.f_preds_17 + S.f_preds_18*Q.f_preds_18 + S.f_preds_19*Q.f_preds_19 + S.f_preds_20*Q.f_preds_20 + S.f_preds_21*Q.f_preds_21 + S.f_preds_22*Q.f_preds_22 + S.f_preds_23*Q.f_preds_23 + S.f_preds_24*Q.f_preds_24 + S.f_preds_25*Q.f_preds_25 + S.f_preds_26*Q.f_preds_26 + S.f_preds_27*Q.f_preds_27 + S.f_preds_28*Q.f_preds_28 + S.f_preds_29*Q.f_preds_29 + S.f_preds_30*Q.f_preds_30 + S.f_preds_31*Q.f_preds_31 + S.f_preds_32*Q.f_preds_32 + S.f_preds_33*Q.f_preds_33 + S.f_preds_34*Q.f_preds_34 + S.f_preds_35*Q.f_preds_35 + S.f_preds_36*Q.f_preds_36 + S.f_preds_37*Q.f_preds_37 + S.f_preds_38*Q.f_preds_38 + S.f_preds_39*Q.f_preds_39 + S.f_preds_40*Q.f_preds_40 + S.f_preds_41*Q.f_preds_41 + S.f_preds_42*Q.f_preds_42 + S.f_preds_43*Q.f_preds_43 + S.f_preds_44*Q.f_preds_44 + S.f_preds_45*Q.f_preds_45 + S.f_preds_46*Q.f_preds_46 + S.f_preds_47*Q.f_preds_47 + S.f_preds_48*Q.f_preds_48 + S.f_preds_49*Q.f_preds_49 + S.f_preds_50*Q.f_preds_50 + S.f_preds_51*Q.f_preds_51 + S.f_preds_52*Q.f_preds_52 + S.f_preds_53*Q.f_preds_53 + S.f_preds_54*Q.f_preds_54 + S.f_preds_55*Q.f_preds_55 + S.f_preds_56*Q.f_preds_56 + S.f_preds_57*Q.f_preds_57 + S.f_preds_58*Q.f_preds_58 + S.f_preds_59*Q.f_preds_59 + S.f_preds_60*Q.f_preds_60 + S.f_preds_61*Q.f_preds_61 + S.f_preds_62*Q.f_preds_62 + S.f_preds_63*Q.f_preds_63 + S.f_preds_64*Q.f_preds_64 + S.f_preds_65*Q.f_preds_65 + S.f_preds_66*Q.f_preds_66 + S.f_preds_67*Q.f_preds_67 + S.f_preds_68*Q.f_preds_68 + S.f_preds_69*Q.f_preds_69 + S.f_preds_70*Q.f_preds_70 + S.f_preds_71*Q.f_preds_71 + S.f_preds_72*Q.f_preds_72 + S.f_preds_73*Q.f_preds_73 + S.f_preds_74*Q.f_preds_74 + S.f_preds_75*Q.f_preds_75 + S.f_preds_76*Q.f_preds_76 + S.f_preds_77*Q.f_preds_77 + S.f_preds_78*Q.f_preds_78 + S.f_preds_79*Q.f_preds_79 + S.f_preds_80*Q.f_preds_80 + S.f_preds_81*Q.f_preds_81 + S.f_preds_82*Q.f_preds_82 + S.f_preds_83*Q.f_preds_83 + S.f_preds_84*Q.f_preds_84 + S.f_preds_85*Q.f_preds_85 + S.f_preds_86*Q.f_preds_86 + S.f_preds_87*Q.f_preds_87 + S.f_preds_88*Q.f_preds_88 + S.f_preds_89*Q.f_preds_89 + S.f_preds_90*Q.f_preds_90 + S.f_preds_91*Q.f_preds_91 + S.f_preds_92*Q.f_preds_92 + S.f_preds_93*Q.f_preds_93 + S.f_preds_94*Q.f_preds_94 + S.f_preds_95*Q.f_preds_95 + S.f_preds_96*Q.f_preds_96 + S.f_preds_97*Q.f_preds_97 + S.f_preds_98*Q.f_preds_98 + S.f_preds_99*Q.f_preds_99 + S.f_preds_100*Q.f_preds_100 + S.f_preds_101*Q.f_preds_101 + S.f_preds_102*Q.f_preds_102 + S.f_preds_103*Q.f_preds_103 + S.f_preds_104*Q.f_preds_104 + S.f_preds_105*Q.f_preds_105 + S.f_preds_106*Q.f_preds_106 + S.f_preds_107*Q.f_preds_107 + S.f_preds_108*Q.f_preds_108 + S.f_preds_109*Q.f_preds_109 + S.f_preds_110*Q.f_preds_110 + S.f_preds_111*Q.f_preds_111 + S.f_preds_112*Q.f_preds_112 + S.f_preds_113*Q.f_preds_113 + S.f_preds_114*Q.f_preds_114 + S.f_preds_115*Q.f_preds_115 + S.f_preds_116*Q.f_preds_116 + S.f_preds_117*Q.f_preds_117 + S.f_preds_118*Q.f_preds_118 + S.f_preds_119*Q.f_preds_119 + S.f_preds_120*Q.f_preds_120 + S.f_preds_121*Q.f_preds_121 + S.f_preds_122*Q.f_preds_122 + S.f_preds_123*Q.f_preds_123 + S.f_preds_124*Q.f_preds_124 + S.f_preds_125*Q.f_preds_125 + S.f_preds_126*Q.f_preds_126 + S.f_preds_127*Q.f_preds_127 + S.f_preds_128*Q.f_preds_128 + S.f_preds_129*Q.f_preds_129 + S.f_preds_130*Q.f_preds_130 + S.f_preds_131*Q.f_preds_131 + S.f_preds_132*Q.f_preds_132 + S.f_preds_133*Q.f_preds_133 + S.f_preds_134*Q.f_preds_134 + S.f_preds_135*Q.f_preds_135 + S.f_preds_136*Q.f_preds_136 + S.f_preds_137*Q.f_preds_137 + S.f_preds_138*Q.f_preds_138 + S.f_preds_139*Q.f_preds_139 + S.f_preds_140*Q.f_preds_140 + S.f_preds_141*Q.f_preds_141 + S.f_preds_142*Q.f_preds_142 + S.f_preds_143*Q.f_preds_143 + S.f_preds_144*Q.f_preds_144 + S.f_preds_145*Q.f_preds_145 + S.f_preds_146*Q.f_preds_146 + S.f_preds_147*Q.f_preds_147 + S.f_preds_148*Q.f_preds_148 + S.f_preds_149*Q.f_preds_149 + S.f_preds_150*Q.f_preds_150 + S.f_preds_151*Q.f_preds_151 + S.f_preds_152*Q.f_preds_152 + S.f_preds_153*Q.f_preds_153 + S.f_preds_154*Q.f_preds_154 + S.f_preds_155*Q.f_preds_155 + S.f_preds_156*Q.f_preds_156 + S.f_preds_157*Q.f_preds_157 + S.f_preds_158*Q.f_preds_158 + S.f_preds_159*Q.f_preds_159 + S.f_preds_160*Q.f_preds_160 + S.f_preds_161*Q.f_preds_161 + S.f_preds_162*Q.f_preds_162 + S.f_preds_163*Q.f_preds_163 + S.f_preds_164*Q.f_preds_164 + S.f_preds_165*Q.f_preds_165 + S.f_preds_166*Q.f_preds_166 + S.f_preds_167*Q.f_preds_167 + S.f_preds_168*Q.f_preds_168 + S.f_preds_169*Q.f_preds_169 + S.f_preds_170*Q.f_preds_170 + S.f_preds_171*Q.f_preds_171 + S.f_preds_172*Q.f_preds_172 + S.f_preds_173*Q.f_preds_173 + S.f_preds_174*Q.f_preds_174 + S.f_preds_175*Q.f_preds_175 + S.f_preds_176*Q.f_preds_176 + S.f_preds_177*Q.f_preds_177 + S.f_preds_178*Q.f_preds_178 + S.f_preds_179*Q.f_preds_179 + S.f_preds_180*Q.f_preds_180 + S.f_preds_181*Q.f_preds_181 + S.f_preds_182*Q.f_preds_182 + S.f_preds_183*Q.f_preds_183 + S.f_preds_184*Q.f_preds_184 + S.f_preds_185*Q.f_preds_185 + S.f_preds_186*Q.f_preds_186 + S.f_preds_187*Q.f_preds_187 + S.f_preds_188*Q.f_preds_188 + S.f_preds_189*Q.f_preds_189 + S.f_preds_190*Q.f_preds_190 + S.f_preds_191*Q.f_preds_191 + S.f_preds_192*Q.f_preds_192 + S.f_preds_193*Q.f_preds_193 + S.f_preds_194*Q.f_preds_194 + S.f_preds_195*Q.f_preds_195 + S.f_preds_196*Q.f_preds_196 + S.f_preds_197*Q.f_preds_197 + S.f_preds_198*Q.f_preds_198 + S.f_preds_199*Q.f_preds_199 + S.f_preds_200*Q.f_preds_200 + S.f_preds_201*Q.f_preds_201 + S.f_preds_202*Q.f_preds_202 + S.f_preds_203*Q.f_preds_203 + S.f_preds_204*Q.f_preds_204 + S.f_preds_205*Q.f_preds_205 + S.f_preds_206*Q.f_preds_206 + S.f_preds_207*Q.f_preds_207 + S.f_preds_208*Q.f_preds_208 + S.f_preds_209*Q.f_preds_209 + S.f_preds_210*Q.f_preds_210 + S.f_preds_211*Q.f_preds_211 + S.f_preds_212*Q.f_preds_212 + S.f_preds_213*Q.f_preds_213 + S.f_preds_214*Q.f_preds_214 + S.f_preds_215*Q.f_preds_215 + S.f_preds_216*Q.f_preds_216 + S.f_preds_217*Q.f_preds_217 + S.f_preds_218*Q.f_preds_218 + S.f_preds_219*Q.f_preds_219 + S.f_preds_220*Q.f_preds_220 + S.f_preds_221*Q.f_preds_221 + S.f_preds_222*Q.f_preds_222 + S.f_preds_223*Q.f_preds_223 + S.f_preds_224*Q.f_preds_224 + S.f_preds_225*Q.f_preds_225 + S.f_preds_226*Q.f_preds_226 + S.f_preds_227*Q.f_preds_227 + S.f_preds_228*Q.f_preds_228 + S.f_preds_229*Q.f_preds_229 + S.f_preds_230*Q.f_preds_230 + S.f_preds_231*Q.f_preds_231 + S.f_preds_232*Q.f_preds_232 + S.f_preds_233*Q.f_preds_233 + S.f_preds_234*Q.f_preds_234 + S.f_preds_235*Q.f_preds_235 + S.f_preds_236*Q.f_preds_236 + S.f_preds_237*Q.f_preds_237 + S.f_preds_238*Q.f_preds_238 + S.f_preds_239*Q.f_preds_239 + S.f_preds_240*Q.f_preds_240 + S.f_preds_241*Q.f_preds_241 + S.f_preds_242*Q.f_preds_242 + S.f_preds_243*Q.f_preds_243 + S.f_preds_244*Q.f_preds_244 + S.f_preds_245*Q.f_preds_245 + S.f_preds_246*Q.f_preds_246 + S.f_preds_247*Q.f_preds_247 + S.f_preds_248*Q.f_preds_248 + S.f_preds_249*Q.f_preds_249 + S.f_preds_250*Q.f_preds_250 + S.f_preds_251*Q.f_preds_251 + S.f_preds_252*Q.f_preds_252 + S.f_preds_253*Q.f_preds_253 + S.f_preds_254*Q.f_preds_254 + S.f_preds_255*Q.f_preds_255 + S.f_preds_256*Q.f_preds_256 + S.f_preds_257*Q.f_preds_257 + S.f_preds_258*Q.f_preds_258 + S.f_preds_259*Q.f_preds_259 + S.f_preds_260*Q.f_preds_260 + S.f_preds_261*Q.f_preds_261 + S.f_preds_262*Q.f_preds_262 + S.f_preds_263*Q.f_preds_263 + S.f_preds_264*Q.f_preds_264 + S.f_preds_265*Q.f_preds_265 + S.f_preds_266*Q.f_preds_266 + S.f_preds_267*Q.f_preds_267 + S.f_preds_268*Q.f_preds_268 + S.f_preds_269*Q.f_preds_269 + S.f_preds_270*Q.f_preds_270 + S.f_preds_271*Q.f_preds_271 + S.f_preds_272*Q.f_preds_272 + S.f_preds_273*Q.f_preds_273 + S.f_preds_274*Q.f_preds_274 + S.f_preds_275*Q.f_preds_275 + S.f_preds_276*Q.f_preds_276 + S.f_preds_277*Q.f_preds_277 + S.f_preds_278*Q.f_preds_278 + S.f_preds_279*Q.f_preds_279 + S.f_preds_280*Q.f_preds_280 + S.f_preds_281*Q.f_preds_281 + S.f_preds_282*Q.f_preds_282 + S.f_preds_283*Q.f_preds_283 + S.f_preds_284*Q.f_preds_284 + S.f_preds_285*Q.f_preds_285 + S.f_preds_286*Q.f_preds_286 + S.f_preds_287*Q.f_preds_287 + S.f_preds_288*Q.f_preds_288 + S.f_preds_289*Q.f_preds_289 + S.f_preds_290*Q.f_preds_290 + S.f_preds_291*Q.f_preds_291 + S.f_preds_292*Q.f_preds_292 + S.f_preds_293*Q.f_preds_293 + S.f_preds_294*Q.f_preds_294 + S.f_preds_295*Q.f_preds_295 + S.f_preds_296*Q.f_preds_296 + S.f_preds_297*Q.f_preds_297 + S.f_preds_298*Q.f_preds_298 + S.f_preds_299*Q.f_preds_299 + S.f_preds_300*Q.f_preds_300 + S.f_preds_301*Q.f_preds_301 + S.f_preds_302*Q.f_preds_302 + S.f_preds_303*Q.f_preds_303 + S.f_preds_304*Q.f_preds_304 + S.f_preds_305*Q.f_preds_305 + S.f_preds_306*Q.f_preds_306 + S.f_preds_307*Q.f_preds_307 + S.f_preds_308*Q.f_preds_308 + S.f_preds_309*Q.f_preds_309 + S.f_preds_310*Q.f_preds_310 + S.f_preds_311*Q.f_preds_311 + S.f_preds_312*Q.f_preds_312 + S.f_preds_313*Q.f_preds_313 + S.f_preds_314*Q.f_preds_314 + S.f_preds_315*Q.f_preds_315 + S.f_preds_316*Q.f_preds_316 + S.f_preds_317*Q.f_preds_317 + S.f_preds_318*Q.f_preds_318 + S.f_preds_319*Q.f_preds_319 + S.f_preds_320*Q.f_preds_320 + S.f_preds_321*Q.f_preds_321 + S.f_preds_322*Q.f_preds_322 + S.f_preds_323*Q.f_preds_323 + S.f_preds_324*Q.f_preds_324 + S.f_preds_325*Q.f_preds_325 + S.f_preds_326*Q.f_preds_326 + S.f_preds_327*Q.f_preds_327 + S.f_preds_328*Q.f_preds_328 + S.f_preds_329*Q.f_preds_329 + S.f_preds_330*Q.f_preds_330) / S.norm_f_preds / Q.norm_f_preds as d_pc
    from search_recs S
      join (select * from search_recs where source_id = 'xc:416410') Q on true --# 1 row, for dot(S,Q)
      join (
        select column1 as species, column2 as slp from (values ('BRCR', 2.531013988420692), ('BEWR', 2.532770076870172), ('LISP', 2.612302265470383), ('OATI', 2.6209269915633775), ('YBCH', 2.693449148368873), ('RWBL', 2.85915840716264), ('YHBL', 2.9407066815495098), ('JUTI', 3.1964930770861226), ('SOSP', 3.3536266585633028), ('EUST', 3.4345511861796005))
        --# select column1 as species, column2 as slp from (values ('BRCR', 2.531013988420692), ('BEWR', 2.532770076870172), ('LISP', 2.612302265470383), ('OATI', 2.6209269915633775), ('YBCH', 2.693449148368873), ('RWBL', 2.85915840716264), ('YHBL', 2.9407066815495098), ('JUTI', 3.1964930770861226), ('SOSP', 3.3536266585633028), ('EUST', 3.4345511861796005), ('WIFL', 3.4793422717445033), ('ABTO', 3.496172099469417), ('GRVI', 3.5193633517510596), ('RCKI', 3.6404688761047668), ('LASP', 3.6512290161829797), ('GTGR', 3.9715303202185503), ('WEVI', 4.211134790180211), ('LOSH', 4.242274068412453), ('MAWA', 4.350559483605008), ('WIWR', 4.367698139109152), ('MOWA', 4.452379012375921), ('SWTH', 4.460531357394325), ('HUVI', 4.645053683966041), ('CBTH', 4.686734006282624), ('GCSP', 4.706664237639098), ('COYE', 4.738316347112597), ('SUTA', 4.768478476743198), ('CACW', 4.897283943714701), ('BTGN', 5.055958148609322), ('CITE', 5.13783401208553), ('PARE', 5.1870673093210415), ('SPTO', 5.289424823342004), ('HOWR', 5.313536619683756), ('VIRA', 5.359643554296269), ('NOMO', 5.435658780172663), ('STJA', 5.443618350266081), ('NOWA', 5.571608467440403), ('TOWA', 5.624281844181796), ('GTTO', 5.672155589635655), ('HOOR', 5.754228876946757), ('BRBL', 5.803098377199502), ('BUOR', 5.860230662760299), ('WETA', 5.870270129701547), ('SAVS', 5.9350801202552175), ('GRRO', 6.088533962280162), ('CASJ', 6.16486681026883), ('EUCD', 6.263018618968476), ('BHCO', 6.2676917297004735), ('GAQU', 6.277149956237753), ('PRAW', 6.322347171125688), ('MAWR', 6.394385748801781), ('BLTE', 6.495218547138002), ('LEBI', 6.50714035895642), ('HOGR', 6.519299953142001), ('GRHE', 6.594901668884092), ('RIRA', 6.778172676625919), ('CEDW', 6.792401744450868), ('BEVI', 6.8168670075358735), ('AMRE', 6.8434601571283), ('PUFI', 6.8800714610394955), ('TRES', 6.932283234725144), ('CAKI', 6.98838225278954), ('LBWO', 7.0568784274031495), ('MALL', 7.058447717494623), ('ISSJ', 7.094417567384517), ('GRCA', 7.104691391888663), ('DOWO', 7.233165856131655), ('RTHA', 7.320819893142206), ('REVI', 7.323189558427389), ('CANW', 7.340249438571233), ('FOSP', 7.341033665941331), ('BTSP', 7.379653544270154), ('CAGU', 7.4009806956460285), ('BCSP', 7.450540318791626), ('CLGR', 7.529309001424839), ('CALT', 7.665288650261056), ('AMBI', 7.670159206932208), ('WCSP', 7.677722963732932), ('CONI', 7.7165008502646755), ('WISA', 7.737650989254115), ('SCOR', 7.7756358010811), ('ROWR', 7.800027821612021), ('VEFL', 7.836716430514252), ('LUWA', 7.861275711075517), ('YGVI', 8.032603594833692), ('EAPH', 8.045889673331839), ('RECR', 8.054306272386695), ('PLVI', 8.0573258089993), ('VGSW', 8.150851135605016), ('DEJU', 8.177422672877068), ('WEKI', 8.192093961397726), ('AMRO', 8.207358641896416), ('COKI', 8.22691108048428), ('WOSJ', 8.23519637345769), ('TBKI', 8.248404895133326), ('BRSP', 8.260137874042956), ('GRAJ', 8.319454820372373), ('CLNU', 8.41210360706917), ('HEWA', 8.443646487477599), ('YBCU', 8.455966185801852), ('GBHE', 8.470547027359219), ('COBH', 8.473457680601122), ('WOTH', 8.47425251776894), ('CORA', 8.504145101647248), ('WFIB', 8.558195246726559), ('GCKI', 8.675845206307086), ('SNEG', 8.71489775217107), ('CAVI', 8.824804994737796), ('BCFL', 8.865598141201284), ('GADW', 8.881805163459138), ('PBGR', 8.883883139405508), ('OCWA', 8.91571997772636), ('BGGN', 8.979557632540418), ('PISI', 9.013475155286066), ('BHGR', 9.048014026311973), ('RNPH', 9.106074085564245), ('CAGN', 9.194456615444194), ('WESA', 9.253091637619606), ('HOSP', 9.329810940921154), ('YBMA', 9.331748009232133), ('HOFI', 9.352768888613367), ('WEGR', 9.36361691338387), ('ATFL', 9.395993001431028), ('OSFL', 9.43129582619472), ('BANO', 9.440415528979932), ('BARS', 9.468711682387298), ('NRWS', 9.483742792319145), ('DUFL', 9.494884209667863), ('BLGR', 9.506186466177965), ('WREN', 9.588832680404476), ('WHIM', 9.59023830590102), ('RCSP', 9.612597665765831), ('BCHU', 9.636988913398117), ('RNGR', 9.649324364699009), ('NAWA', 9.65689968046352), ('WEWP', 9.677391716081651), ('GIWO', 9.683855960858526), ('CAQU', 9.68931968940774), ('TRBL', 9.69457759244987), ('AMPI', 9.705954469663716), ('CATH', 9.756499339000824), ('FOTE', 9.786710739159583), ('INDO', 9.798281775522625), ('MGWA', 9.850680760487974), ('WAVI', 9.87826785453523), ('INBU', 9.973553500978726), ('BNST', 10.003575107070436), ('GWTE', 10.004672621664334), ('CATE', 10.024939059789673), ('COCK', 10.058411906242194), ('BRAN', 10.070341811872169), ('LBCU', 10.07268529924788), ('BEKI', 10.08892382643895), ('GREG', 10.12263126882629), ('GRFL', 10.225121670747903), ('MOCH', 10.288602435960087), ('AMAV', 10.289414125141299), ('NOHA', 10.296462703885892), ('VERD', 10.359538873224341), ('RWBU', 10.38093207690275), ('BTAH', 10.444779583769158), ('BOGU', 10.459158320788868), ('BTPI', 10.46969181032973), ('SATH', 10.494303412559143), ('EWPW', 10.499739379193759), ('BBWA', 10.528161986296945), ('AMCR', 10.534259580501928), ('ACWO', 10.556080342422899), ('LETE', 10.676353186733806), ('HETH', 10.703737887764852), ('YRWA', 10.740663773204677), ('BETH', 10.75306310082828), ('BESP', 10.775242600307786), ('JAWE', 10.79885071045656), ('LBDO', 10.881159712750236), ('CASP', 10.881791795833225), ('PIWA', 10.917633353380774), ('LENI', 11.001262318008), ('WIWA', 11.076173104920157), ('ELTE', 11.134156481436278), ('BTYW', 11.159490114984676), ('MEGU', 11.19561006888942), ('COGA', 11.234191400659466), ('SBMU', 11.272577125505444), ('GWFG', 11.283658145820533), ('HOLA', 11.289166873118502), ('YHPA', 11.30167898908479), ('BLPH', 11.328764066954324), ('COHA', 11.399349884857482), ('GRSP', 11.426447575635958), ('ROSA', 11.514579864617541), ('WHWO', 11.554579673748899), ('BCNH', 11.616288770600107), ('WEWA', 11.63805110813809), ('COGD', 11.660989309708581), ('WTSP', 11.705153274178675), ('YFGU', 11.710106064272063), ('RMPA', 11.745186647054975), ('WTKI', 11.745444015371856), ('DCFL', 11.775270191659672), ('SPSA', 11.8079276487243), ('WILL', 11.820715335632878), ('ANHU', 11.874078168204854), ('RBNU', 11.951802966228469), ('HAWO', 12.002295226387616), ('LEFL', 12.040031240389034), ('LAZB', 12.07708553255923), ('NAPA', 12.081497967472878), ('CSWA', 12.145381176599757), ('SOGR', 12.202756337377206), ('LAGO', 12.2319360538941), ('HAFL', 12.253908421142965), ('SOSA', 12.280515337039642), ('HOWA', 12.309078288331046), ('CAFI', 12.342163223335955), ('REDH', 12.342852696646961), ('CAHU', 12.34437175640791), ('PIGR', 12.370777588356646), ('AMCO', 12.371449846832995), ('CRTH', 12.40621727101908), ('CCSP', 12.418824267906743), ('WEME', 12.461447585447527), ('NUWO', 12.494546256185368), ('WBNU', 12.550259198156121), ('AMKE', 12.5879705692463), ('BFAL', 12.599010072947978), ('SUSC', 12.607163481842957), ('LEGO', 12.610194266715151), ('EGGO', 12.717279991381758), ('BTMJ', 12.734900484409916), ('LCPA', 12.785958480665037), ('SAPH', 12.85096814906219), ('BBWO', 12.853764320741432), ('TOSO', 12.887230564749972), ('PSFL', 12.887776107638214), ('BUSH', 12.89277208362413), ('WITU', 12.947646911113521), ('SNPL', 12.950877795143759), ('PAWR', 13.015895644703479), ('EVGR', 13.030762053934088), ('AMWI', 13.092302802697999), ('RBSA', 13.092911293379633), ('PTWH', 13.151428283688343), ('RBGR', 13.23211415043763), ('TRKI', 13.35721436984883), ('SEPL', 13.39987526822362), ('RCPA', 13.534886516815417), ('NOCA', 13.557554884405473), ('BLTU', 13.641568590587058), ('MAGO', 13.663751672726363), ('PHAI', 13.679443332396383), ('CANG', 13.712275984543163), ('PIJA', 13.779124742830192), ('WWDO', 13.93986752648307), ('NOFL', 13.949757055334633), ('NOPA', 13.967616353635064), ('FLOW', 14.046397653533175), ('AMGO', 14.09764953376981), ('MOUQ', 14.180958040833765), ('GRWA', 14.244986224414498), ('RTLO', 14.427145225017854), ('SACR', 14.49586095683052), ('LCTH', 14.535235838532195), ('RTPI', 14.566881452855002), ('COHU', 14.658086425044775), ('WTSW', 14.69155469601567), ('VATH', 14.760312386336937), ('CACG', 14.8905103260917), ('WEBL', 14.919877548339713), ('RRPA', 14.92159708600756), ('RSHA', 15.02440736450781), ('ROYT', 15.034523996218242), ('MWPW', 15.101850408480322), ('DUNL', 15.112934844811912), ('CLSW', 15.228457857956569), ('CHSP', 15.325009416549817), ('MERL', 15.36083794907528), ('SWHA', 15.483503064294434), ('SORA', 15.504577349038902), ('AFCD', 15.557755066107376), ('COLO', 15.571589072540275), ('PUMA', 15.580846337217782), ('GHOW', 15.592303703808632), ('LALO', 15.606447644743367), ('RBGU', 15.626537739256971), ('VASW', 15.65960731659203), ('DCCO', 15.705802157823962), ('CBCH', 15.724589490413988), ('PYNU', 15.742447906804415), ('GWGU', 16.06143149450292), ('PAGP', 16.112702834354685), ('SURF', 16.159712354031832), ('MOBL', 16.38585694486965), ('MODO', 16.57150675239372), ('PEFA', 16.61430653763232), ('SABS', 16.619973334139644), ('yecpar', 16.800916569278108), ('ALHU', 16.815754768941343), ('LEOW', 16.848703315122915), ('VESP', 16.867215051072662), ('PECO', 16.904050667056193), ('BUOW', 16.982252032483235), ('WEGU', 17.306432365345316), ('SBDO', 17.61577088023981), ('BLRA', 17.704378641694433), ('SNGO', 17.896372239524084), ('OSPR', 17.91787001874239), ('COPO', 18.000715626979783), ('DUWA', 18.28735100533962), ('PIWO', 18.399724416044027), ('GBTE', 18.476700211882985), ('BUFF', 18.570837432539953), ('KILL', 18.62114372213435), ('HEEG', 18.641071653818972), ('BLSK', 18.673321079568804), ('RODO', 18.96827923990534), ('NSWO', 19.09820594164006), ('WATA', 19.209856327280047), ('BLOY', 19.4653992890448), ('GRYE', 19.48918754998192), ('LESA', 19.491976161971518), ('LAGU', 19.787924146071667), ('AMDI', 19.90295247329546), ('MAMU', 20.236128093649693), ('WESO', 20.6932070454643), ('COMU', 21.32292012451962), ('SPOW', 22.169901163965974), ('ELOW', 22.460205414042218), ('TUSW', 22.492817124183066), ('BBPL', 22.779129209772936), ('BRAC', 23.61234120289399))
      ) slp on S.species = slp.species
    where true
      --# Empty subquery for species outside of placeFilter
      and S.quality in ('A', 'B')
      and S.source_id != 'xc:416410' --# Exclude query_rec from results
  ) S
) S
where
  sp_d_pc_i <= 3
order by
  slp asc, d_pc asc
limit 31